## Risk-adjusted mean consumption growth, MC method

In [26]:
%run src/lg_model.py

In [27]:
lg = LinearGaussian(γ=7.5)

Here's the spectral radius at default values:

In [66]:
lrm_analytic(lg)

1.0004504439817938

In [67]:
lrm_discretized(lg)

1.000473816505887

In [68]:
se_mc = lrm_mc_factory(lg, parallel_flag=False)

In [69]:
%time se_mc(m=1500, n=1000)

CPU times: user 267 ms, sys: 3.26 ms, total: 270 ms
Wall time: 271 ms


1.0006921558512156

Once more with parallelization:

In [70]:
se_mc = lrm_mc_factory(lg, parallel_flag=True)

In [71]:
%time se_mc(m=1500, n=1000)

CPU times: user 495 ms, sys: 6.55 ms, total: 502 ms
Wall time: 409 ms


1.0003500545047028

For the stability exponent $\Lambda$, with parameter vals from SSY

In [72]:
lg.β * lrm_analytic(lg)**(1 - 1/lg.ψ)

0.9981498252042409

Generate the table

In [73]:
n_vals = np.array([250, 500, 750])
m_vals = np.array([1000, 2000, 3000, 4000, 5000])

In [74]:
k = 100
MC_draws = np.empty(k)
Lambda_draws = np.empty(k)

MC_means = np.empty((len(n_vals), len(m_vals)))
MC_stds = np.empty((len(n_vals), len(m_vals)))
Lambda_means = np.empty((len(n_vals), len(m_vals)))
Lambda_stds = np.empty((len(n_vals), len(m_vals)))

for n_i, n in enumerate(n_vals):
    for m_i, m in enumerate(m_vals):
        print(f'Calculating n={n}, m={m}')
        
        for i in range(k):
            MC_draws[i] = se_mc(n=n, m=m)
        
        Lambda_draws = lg.β * MC_draws**(1 - 1/lg.ψ)
        
        MC_means[n_i, m_i] = MC_draws.mean()
        MC_stds[n_i, m_i] = MC_draws.std()
        
        Lambda_means[n_i, m_i] = Lambda_draws.mean()
        Lambda_stds[n_i, m_i] = Lambda_draws.std()

Calculating n=250, m=1000
Calculating n=250, m=2000
Calculating n=250, m=3000
Calculating n=250, m=4000
Calculating n=250, m=5000
Calculating n=500, m=1000
Calculating n=500, m=2000
Calculating n=500, m=3000
Calculating n=500, m=4000
Calculating n=500, m=5000
Calculating n=750, m=1000
Calculating n=750, m=2000
Calculating n=750, m=3000
Calculating n=750, m=4000
Calculating n=750, m=5000


In [75]:
def make_table(means, stds):
    means_strings = means.round(7).astype(str)
    n_strings = n_vals.astype(str)

    start_table = r"""
    \begin{table}
    \centering
    \begin{tabular}{llll}
    """

    m_table = ' & m = '.join(m_vals.astype(str))
    m_table = '    & m = ' + m_table + r' \\' + '\n' + r'\hline \hline' '\n'

    end_table = r"""
    \end{tabular}
    \end{table}
    """

    row_string = ''
    for row in range(len(n_strings)):
        temp_means = ' & '.join(means_strings[row, :])

        x = ['{:f}'.format(item) for item in stds[row, :]]    
        temp_stds = '(' + ') & ('.join(x) + ')'
        row_string += f'n = {n_strings[row]} & ' + temp_means + r' \\' + '\n'
        row_string += '         & ' + temp_stds + r' \\' + '\n'
        row_string += r'\hline' '\n'

    print(start_table + m_table + row_string + end_table)

In [76]:
make_table(MC_means, MC_stds)


    \begin{table}
    \centering
    \begin{tabular}{llll}
        & m = 1000 & m = 2000 & m = 3000 & m = 4000 & m = 5000 \\
\hline \hline
n = 250 & 1.000694 & 1.0006905 & 1.000694 & 1.0006932 & 1.0006934 \\
         & (0.000076) & (0.000048) & (0.000032) & (0.000037) & (0.000029) \\
\hline
n = 500 & 1.0006208 & 1.0006091 & 1.0005813 & 1.0005733 & 1.0005775 \\
         & (0.000084) & (0.000066) & (0.000068) & (0.000084) & (0.000062) \\
\hline
n = 750 & 1.0005979 & 1.0005762 & 1.0005764 & 1.0005611 & 1.0005523 \\
         & (0.000112) & (0.000096) & (0.000076) & (0.000074) & (0.000092) \\
\hline

    \end{tabular}
    \end{table}
    


In [77]:
make_table(Lambda_means, Lambda_stds)


    \begin{table}
    \centering
    \begin{tabular}{llll}
        & m = 1000 & m = 2000 & m = 3000 & m = 4000 & m = 5000 \\
\hline \hline
n = 250 & 0.9982308 & 0.9982297 & 0.9982308 & 0.9982305 & 0.9982306 \\
         & (0.000025) & (0.000016) & (0.000011) & (0.000012) & (0.000010) \\
\hline
n = 500 & 0.9982065 & 0.9982026 & 0.9981934 & 0.9981907 & 0.9981921 \\
         & (0.000028) & (0.000022) & (0.000022) & (0.000028) & (0.000021) \\
\hline
n = 750 & 0.9981989 & 0.9981916 & 0.9981917 & 0.9981866 & 0.9981837 \\
         & (0.000037) & (0.000032) & (0.000025) & (0.000025) & (0.000031) \\
\hline

    \end{tabular}
    \end{table}
    


In [78]:
lg = LinearGaussian(γ=7.5)